##### Author: Bishesh Kafle
##### Date : 2024-07-22

In [1]:
from kafka import KafkaProducer
import time
from utils.mysql_connection import *
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
import os
from dotenv import load_dotenv

load_dotenv('../config/.env')

True

In [2]:
# Load stream_data table form kafka_prod database
df = table_df('kafka_prod','stream_data')
# Add an order_id column with unique identifiers for each row
df['order_id'] = np.arange(len(df))

/Users/bses/F1_Intern/Kafka_Streaming/utils/mysql_connection.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=query, con=con.connection)


In [3]:
# Create a SparkSession
spark = SparkSession.builder \
    .appName("Kafka Streaming") \
    .getOrCreate()

24/07/24 09:26:20 WARN Utils: Your hostname, Bisheshs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.13.166.144 instead (on interface en0)
24/07/24 09:26:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/24 09:26:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark_df = spark.createDataFrame(df)

In [5]:
spark_df.printSchema()

root
 |-- tran_date: string (nullable = true)
 |-- account_number: string (nullable = true)
 |-- branch: string (nullable = true)
 |-- product: string (nullable = true)
 |-- lcy_amount: string (nullable = true)
 |-- transaction_code: string (nullable = true)
 |-- description1: string (nullable = true)
 |-- dc_indicator: string (nullable = true)
 |-- is_salary: string (nullable = true)
 |-- order_id: long (nullable = true)



In [ ]:
# Assign Kafka topic name and bootstrap server address
KAFKA_TOPIC_NAME_CONS = "Topic2"
KAFKA_BOOTSTRAP_SERVERS_CONS = 'localhost:9092'
KEY = os.getenv('KEY')
# Start Producer Application
if __name__ == "__main__":
    print("Kafka Producer Application Started ... ")

    # Initialize KafkaProducer object with specified bootstrap server and value serializer
    kafka_producer_obj = KafkaProducer(
        bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS_CONS,
        value_serializer=lambda x: x.encode('utf-8')
    )
    spark_df = spark_df.withColumn('encrypted_account_number', expr(f"base64(aes_encrypt(account_number, '{KEY}' , 'ECB', 'PKCS'))"))

    # Convert the DataFrame to a list of dictionaries, each representing a record
    data = spark_df.toPandas().to_dict(orient="records")
       
    # Iterate through each record in the data
    for message in data:
        # Create a list and store field values of the message
        message_fields_value_list = [message["order_id"], message["encrypted_account_number"], message["branch"], message["transaction_code"]]

        # Concatenate field values into a single string separated by commas
        message = ','.join(str(v) for v in message_fields_value_list)

        # Print the type and content of the message for debugging purposes
        print("Message Type: ", type(message))
        print("Message: ", message)

        # Send the message to the specified Kafka topic
        kafka_producer_obj.send(KAFKA_TOPIC_NAME_CONS, message)

        # Pause for a short time before sending the next message
        time.sleep(1)

    print("Kafka Producer Application Completed. ")
    

Kafka Producer Application Started ... 


Message Type:  <class 'str'>
Message:  0,LLY2iKGyxElG5+2r0eurWcPPH8xV0PpM3ad6GUa9LmI=,15,CI
Message Type:  <class 'str'>
Message:  1,LLY2iKGyxElG5+2r0eurWcPPH8xV0PpM3ad6GUa9LmI=,15,CI
Message Type:  <class 'str'>
Message:  2,E97z6MmS7wv2ZB4L/z/jqnPyoXhfIacThZ+ov5upc3M=,15,CI
Message Type:  <class 'str'>
Message:  3,E97z6MmS7wv2ZB4L/z/jqnPyoXhfIacThZ+ov5upc3M=,15,CI
Message Type:  <class 'str'>
Message:  4,E97z6MmS7wv2ZB4L/z/jqnPyoXhfIacThZ+ov5upc3M=,15,CI
Message Type:  <class 'str'>
Message:  5,RaOXBO26LRxTdo9rDMiYtXzd5pTTGC6MMp9M5rpy2n0=,15,CI
Message Type:  <class 'str'>
Message:  6,RaOXBO26LRxTdo9rDMiYtaEEsmHRJc/qBmMDCjDDC78=,15,CI
Message Type:  <class 'str'>
Message:  7,IePg4bkHt0xLcjga9BDXpSVpeEg4tUdytwyhkv4NxNQ=,15,CI


24/07/24 09:26:32 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


Message Type:  <class 'str'>
Message:  8,IePg4bkHt0xLcjga9BDXpSVpeEg4tUdytwyhkv4NxNQ=,15,CI
Message Type:  <class 'str'>
Message:  9,IePg4bkHt0xLcjga9BDXpSVpeEg4tUdytwyhkv4NxNQ=,15,CI
Message Type:  <class 'str'>
Message:  10,h6X5wcajik41I7iIjGnltlkSbgUSvQOeHUOUdeBN0ic=,15,CI
Message Type:  <class 'str'>
Message:  11,h6X5wcajik41I7iIjGnltvZe9IyhsoQmdK49Ft7BVQU=,15,CI
Message Type:  <class 'str'>
Message:  12,kUnYJvo5ieLcgzp8+FtBU19yPI8fXgiHX0bNpg80GYc=,15,O
Message Type:  <class 'str'>
Message:  13,kUnYJvo5ieLcgzp8+FtBU0p//YOpvsohVTHhPXO22hw=,15,CI
Message Type:  <class 'str'>
Message:  14,kUnYJvo5ieLcgzp8+FtBUyP2AjbaZzvGcCZDaSwoZhg=,15,CI
Message Type:  <class 'str'>
Message:  15,X0g/k/AsgsFLlzgJ49dlrKEEsmHRJc/qBmMDCjDDC78=,15,CI
Message Type:  <class 'str'>
Message:  16,kUnYJvo5ieLcgzp8+FtBU4N0FZNup0KJvxgS5276ov4=,15,CI
Message Type:  <class 'str'>
Message:  17,kUnYJvo5ieLcgzp8+FtBU4N0FZNup0KJvxgS5276ov4=,15,CI
Message Type:  <class 'str'>
Message:  18,kUnYJvo5ieLcgzp8+FtBU2wiggXAwAi